<a href="https://colab.research.google.com/github/gkmurphy/PUS2024_GMurphy/blob/main/HW5_GwenMurphy_RandomForestipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following the paper "Examining the feasibility of using open data to benchmark building energy usage in cities: A data science and policy perspective" [Roth et al 2019](https://www.researchgate.net/publication/337074109_Examining_the_feasibility_of_using_open_data_to_benchmark_building_energy_usage_in_cities_A_data_science_and_policy_perspective), choose 2 cities in the dataset available through the paper's github repository https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking

This paper is an excellent example of reproducible research.

I also created an example of using random forest classifier and regressors here https://github.com/fedhere/PUS2022_FBianco/blob/master/classdemo/COVID19LOS_featureExtractionRFexample.ipynb and there are links at the bottom of this notebook to useful functions and examples on the internet of applications of methods you will need to use.

1. Read the introduction to the paper and describe here (briefly) the goals of their analysis (and the analysis that you are about to reproduce). Max 200 words.




> This paper aims builds energy benchmarks by leveraging city-specific, public open data. The effectiveness of this data is evaluated in two benchmarking models—a random forest and a lasso regression. There performance was compared  against models based on the Commercial Building Energy Consumption Survey (CBECS) dataset, which is commonly used for Energy Star ratings. The study finds that models using open data sources from ten major cities significantly outperform those using the CBECS dataset, with 90% of the open data models yielding better results.The analysis identifies building area, property type, conditioned area, and water usage as critical variables for energy benchmarking. The paper argues that open data not only enhances model accuracy but also promotes transparency in the benchmarking process. It advocates for two main changes to current practices: the adoption of guidelines supporting a data-driven framework for benchmarking that utilizes openly available data, and the development of policies that both publicize benchmarking outcomes and incentivize energy efficiency.

2. Choose 2 cities (any 2 cities except Seattle cause I am running some of the Seattle analysis below for guidance)


> Boston & London


3. For each of the 2 cities reproduce their analysis by

  3.1 gathering the original data from their repository (see below for Seattle example)



```
boston = pd.read_csv("https://raw.githubusercontent.com/Urban-Informatics-Lab/Open-Data-Benchmarking/master/Boston/boston_final.csv")
london = pd.read_csv("https://raw.githubusercontent.com/Urban-Informatics-Lab/Open-Data-Benchmarking/master/London/london_final.csv")
```



  3.2 clean the data according to their data preparation scheme, including one-hot-encoding categorical variables, except to impute missing data using KNearestNeighors instead of Gibbs sampling (see below)

  3.3 run a Random Forest (RF) Regressor to predict the total energy consumption.

  3.4 evaluate the RF model performance by printing the R2 score (the default score in SKLearn RF) for training and test set (discuss)

  3.5 plot the features sorted by their importance and identify the most important features. are they consistent between the 2 cities? are they consistent with the paper's result?
4 compare the result for the 2 cities (discuss)

5. Extra credit:

  5.1 modify the target variable to a binary variable choosing the median of the variable values as the the energy threshold and predict whether the value is higher or lower than the threshold
  5.2 Run a Random Forest Classifier to predict if an observation is in the upper 50% percentile or lower 50% percentile (above or below the median threshold)
  5.3 Measure the performance of the model on training and test data
  5.4 Measure the feature importance and assess if the important feature have changed (discuss)

In [ ]:
import pandas as pd
import numpy as np

# choose a city and read in the data

In [ ]:
seattle = pd.read_csv("https://raw.githubusercontent.com/Urban-Informatics-Lab/Open-Data-Benchmarking/master/Seattle/2016_Building_Energy_Benchmarking.csv")

boston = pd.read_csv("https://raw.githubusercontent.com/Urban-Informatics-Lab/Open-Data-Benchmarking/master/Boston/boston_final.csv")
london = pd.read_csv("https://raw.githubusercontent.com/Urban-Informatics-Lab/Open-Data-Benchmarking/master/London/london_final.csv")


# inspect the dataframe

In [ ]:
print("there are (rows, columns) = ", (boston.shape), "in the dataframe")

there are (rows, columns) =  (1599, 16) in the dataframe


In [ ]:
boston.describe()

,Unnamed: 0,gross_area_SF,year_built,h2o_intensity_GALSF,site_eui_KBTUSF,energy_star_score,ghg_emissions_MTCO2e,ghg_intens_KGCO2SF,total_site_energy_KBTU,log_total_site_energy_KBTU,electricity_percentage,gas_percentage
count,1599.000000,1.599000e+03,1599.000000,1.599000e+03,1599.000000,1076.000000,1599.000000,1598.000000,1.599000e+03,1599.000000,1577.000000,1306.000000
mean,901.190119,1.494918e+05,1954.343340,1.673398e+06,82.131207,68.187732,1954.343340,5.547810,1.325400e+07,15.386588,47.041725,56.545865
std,516.225894,2.713492e+05,70.996812,3.822188e+07,56.144612,29.637595,70.996812,3.865617,3.927494e+07,1.434270,29.181977,27.063293
min,3.000000,1.000000e+00,1000.000000,0.000000e+00,1.300000,1.000000,1000.000000,0.000000,1.426290e+04,9.565417,0.000000,0.000000
25%,459.500000,3.880350e+04,1920.000000,1.090000e+01,49.300000,49.750000,1920.000000,3.300000,2.139090e+06,14.575890,22.800000,38.900000
50%,894.000000,7.000000e+04,1969.000000,2.330000e+01,69.000000,78.000000,1969.000000,4.700000,4.840347e+06,15.392497,41.000000,61.350000
75%,1345.500000,1.490590e+05,2000.000000,4.385000e+01,99.200000,93.000000,2000.000000,6.700000,1.191536e+07,16.293339,66.600000,78.500000
max,1799.000000,4.699442e+06,2016.000000,8.829243e+08,323.600000,100.000000,2016.000000,24.300000,9.747548e+08,20.697697,100.000000,100.000000


# TASK 1 Clean the data
follow closely the second paragraph of Section 4 DATA. Name each step you do accordingly to the description in the paragraph. you can also help yourself inspecting the final datasets posted on the github repo but name each action according to the paper narrative: for example, if you drop "Building ID" indicate that this is done as part of
_We then cleaned the datasets by removing irrelevant building-identifying features (such as
address)_, or if dropping a variable with too many missing values _removing features that had missing values for greater than 40% of the buildings in the dataset._

https://www.researchgate.net/publication/337074109_Examining_the_feasibility_of_using_open_data_to_benchmark_building_energy_usage_in_cities_A_data_science_and_policy_perspective

In [ ]:
# summarize the number of rows with missing values for each column
print('Boston Data: \n')

for c in boston.columns:

  # count number of rows with missing values
  n_miss = boston[c].isnull().sum()
  perc = n_miss / boston.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, boston[c].dtype,
                                                  n_miss, perc))

print('\n')
print('London Data: \n')

for c in london.columns:

  # count number of rows with missing values
  n_miss = london[c].isnull().sum()
  perc = n_miss / london.shape[0] * 100
  print('%s (%s):  Missing: %d (%.1f%%)' % (c, london[c].dtype,
                                                  n_miss, perc))


Boston Data: 

Unnamed: 0 (int64):  Missing: 0 (0.0%)
reported (object):  Missing: 0 (0.0%)
property_type (object):  Missing: 0 (0.0%)
gross_area_SF (float64):  Missing: 0 (0.0%)
year_built (int64):  Missing: 0 (0.0%)
h2o_intensity_GALSF (float64):  Missing: 0 (0.0%)
zip (object):  Missing: 0 (0.0%)
property_uses (object):  Missing: 0 (0.0%)
site_eui_KBTUSF (float64):  Missing: 0 (0.0%)
energy_star_score (float64):  Missing: 523 (32.7%)
ghg_emissions_MTCO2e (int64):  Missing: 0 (0.0%)
ghg_intens_KGCO2SF (float64):  Missing: 1 (0.1%)
total_site_energy_KBTU (float64):  Missing: 0 (0.0%)
log_total_site_energy_KBTU (float64):  Missing: 0 (0.0%)
electricity_percentage (float64):  Missing: 22 (1.4%)
gas_percentage (float64):  Missing: 293 (18.3%)


London Data: 

Unnamed: 0 (int64):  Missing: 0 (0.0%)
rrn (object):  Missing: 0 (0.0%)
post_town (object):  Missing: 0 (0.0%)
county (object):  Missing: 26 (0.1%)
zip (int64):  Missing: 0 (0.0%)
energy_rating_band (object):  Missing: 0 (0.0%)
ener

No Data is missing for more than 40% of rows so no cols were dropped.

In [ ]:
for c in boston.columns:

  # count number of rows with missing values
  n_miss = boston[c].isnull().sum()
  perc = n_miss / boston.shape[0] * 100
  if perc > 40:
    boston.drop(c, axis=1, inplace=True)

print("In Boston Data, there are (rows, columns) = ", (boston.shape), "in the dataframe")

for c in london.columns:

  # count number of rows with missing values
  n_miss = london[c].isnull().sum()
  perc = n_miss / london.shape[0] * 100
  if perc > 40:
    boston.drop(c, axis=1, inplace=True)

print("In London Data, there are (rows, columns) = ", (london.shape), "in the dataframe")


In Boston Data, there are (rows, columns) =  (1599, 16) in the dataframe
In London Data, there are (rows, columns) =  (23474, 25) in the dataframe


# TASK 2: transform input feature as needed
for example replace features with log features. Guide yourself with the text, the final dataset names, and also the code here https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd

In [ ]:
boston.describe()
london.describe()


 #what shoudl you look at in the result below?

  #Count - looking for missing data
  #Mean/Med  - central tendancy of data ; some are very close but others like energy rating show skew


,Unnamed: 0,zip,energy_rating,total_CO2_emissions_MT,floor_area_M2,fuel_eui_KWHM2,electricity_eui_KWHM2,typical_fuel_eui_KWHM2,typical_electricity_eui_KWHM2,renewables_percent_thermal,renewables_percent_electrical,total_heat_energy_KWH,total_electricity_KWH,total_heat_energy_typical_KWH,total_electricity_typical_KWH,floor_area_SF,total_site_energy_KBTU,log_total_site_energy_KBTU,site_eui_KBTUSF
count,23474.000000,23474.000000,23474.000000,23474.000000,23474.000000,23474.000000,23474.00000,23474.000000,23474.000000,23474.000000,23474.000000,2.347400e+04,2.347400e+04,2.347400e+04,2.347400e+04,2.347400e+04,2.347400e+04,23474.000000,23474.000000
mean,11737.500000,9271.378887,134.872966,304.372029,3448.422946,169.854861,76.29390,212.057638,65.405129,0.157949,0.424793,6.236950e+05,3.293685e+05,8.143107e+05,2.552551e+05,3.711848e+04,3.251986e+06,14.360666,78.028784
std,6776.504446,5320.827711,581.743761,882.967259,6432.287065,100.733843,55.41504,116.658846,33.401942,2.498934,4.355098,1.913214e+06,1.010481e+06,2.398929e+06,6.078081e+05,6.923649e+04,9.391810e+06,0.947959,39.438203
min,1.000000,1.000000,-1.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.076390e+01,1.269316e+03,7.146234,1.267994
25%,5869.250000,4663.250000,77.000000,78.000000,1322.000000,108.000000,42.00000,154.000000,40.000000,0.000000,0.000000,1.820040e+05,6.684000e+04,2.229379e+05,6.455020e+04,1.422988e+04,9.038492e+05,13.714418,52.304750
50%,11737.500000,9286.500000,95.000000,127.000000,1907.000000,146.000000,58.00000,165.000000,50.000000,0.000000,0.000000,2.990530e+05,1.156260e+05,3.630060e+05,1.106326e+05,2.052676e+04,1.468815e+06,14.199966,66.252684
75%,17605.750000,13841.750000,116.000000,261.000000,3379.750000,203.000000,92.00000,234.000000,83.000000,0.000000,0.000000,5.754570e+05,2.723235e+05,7.364308e+05,2.431839e+05,3.637929e+04,2.883832e+06,14.874630,91.612562
max,23474.000000,18480.000000,9999.000000,34439.000000,280912.000000,746.000000,725.00000,1455.000000,337.000000,90.300000,100.000000,9.905038e+07,3.334334e+07,1.123648e+08,2.556299e+07,3.023709e+06,4.322920e+08,19.884612,256.451775


In [ ]:
boston.info()
london.info()

#why is this helpful to print?
  # to see data type and missing values for each col. We say this above, but this is a quicker way without writing a loop

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  1599 non-null   int64  
 1   reported                    1599 non-null   object 
 2   property_type               1599 non-null   object 
 3   gross_area_SF               1599 non-null   float64
 4   year_built                  1599 non-null   int64  
 5   h2o_intensity_GALSF         1599 non-null   float64
 6   zip                         1599 non-null   object 
 7   property_uses               1599 non-null   object 
 8   site_eui_KBTUSF             1599 non-null   float64
 9   energy_star_score           1076 non-null   float64
 10  ghg_emissions_MTCO2e        1599 non-null   int64  
 11  ghg_intens_KGCO2SF          1598 non-null   float64
 12  total_site_energy_KBTU      1599 non-null   float64
 13  log_total_site_energy_KBTU  1599 

## Throw away data where the **target** variable is missing

I will remove each row where I do not have the target variable. To do that I use `df.dropna()` and the subset argument of `df.dropna()` set to the name of the variable. The `how` should be set to "any" cause you want to drop the row where any values in the subset is NaN (this is a subset of one column, so it is obvious, but imagine if you were passing more than one column to the call `dropna()`

In [ ]:
seattle["SiteEnergyUse(kBtu)"].isna().sum()
boston["total_site_energy_KBTU"].isna().sum()
london["total_site_energy_KBTU"].isna().sum()


#5 missing seattle ; none for my cities

0

In [ ]:
original_len = seattle.shape[0]
seattle.dropna(subset=['SiteEnergyUse(kBtu)'], inplace=True)

print("I lost {:.2f}% of the data in seattle".format((1 - (seattle.shape[0] / original_len)) * 100))


original_len = boston.shape[0]
boston.dropna(subset=['total_site_energy_KBTU'], inplace=True)

print("I lost {:.2f}% of the data in boston".format((1 - (boston.shape[0] / original_len)) * 100))


original_len = london.shape[0]
london.dropna(subset=['total_site_energy_KBTU'], inplace=True)

print("I lost {:.2f}% of the data in london".format((1 - (london.shape[0] / original_len)) * 100))

I lost 0.15% of the data in seattle
I lost 0.00% of the data in boston
I lost 0.00% of the data in london


In [ ]:
# isolate the target variable first (endogenous)
y = seattle['SiteEnergyUse(kBtu)'].values
yboston = boston['total_site_energy_KBTU'].values
ylondon = london['total_site_energy_KBTU'].values

In [ ]:
# and the input variables (exogenous)
X = seattle.drop('SiteEnergyUse(kBtu)', axis=1)
xboston = boston.drop('total_site_energy_KBTU', axis=1)
xlondon = london.drop('total_site_energy_KBTU', axis=1)


### separate categorical and numerical values
you will proceed to missing data imputation differently in the 2 cases

In [ ]:
# I am giving you the solution but please take note of how one does this!
X_numeric_data = X.select_dtypes(include=[np.number])
X_categorical_data = X.select_dtypes(exclude=[np.number])

X_numeric_data_boston = xboston.select_dtypes(include=[np.number])
X_categorical_data_boston = xboston.select_dtypes(exclude=[np.number])

X_numeric_data_london = xlondon.select_dtypes(include=[np.number])
X_categorical_data_london = xlondon.select_dtypes(exclude=[np.number])

In [ ]:
print(X_numeric_data.shape)
print(X_numeric_data_boston.shape)
print(X_numeric_data_london.shape)

(3371, 29)
(1599, 11)
(23474, 18)


In [ ]:
print(X_numeric_data.head())
print(X_numeric_data_boston.head())
print(X_numeric_data_london.head())


## is there anything suspicious? anything thas is in fact **not** a numerical variable? that will depend on how you did in TASK 1
    #there is some missing data

   OSEBuildingID  DataYear  ZipCode  CouncilDistrictCode  Latitude  Longitude  \
0              1      2016  98101.0                    7  47.61220 -122.33799   
1              2      2016  98101.0                    7  47.61317 -122.33393   
2              3      2016  98101.0                    7  47.61393 -122.33810   
3              5      2016  98101.0                    7  47.61412 -122.33664   
4              8      2016  98121.0                    7  47.61375 -122.34047   

   YearBuilt  NumberofBuildings  NumberofFloors  PropertyGFATotal  ...  \
0       1927                1.0              12             88434  ...   
1       1996                1.0              11            103566  ...   
2       1969                1.0              41            956110  ...   
3       1926                1.0              10             61320  ...   
4       1980                1.0              18            175580  ...   

   SourceEUIWN(kBtu/sf)  SiteEnergyUseWN(kBtu)  SteamUse(kBtu)  \
0 

In [ ]:
print(X_categorical_data.shape)
print(X_categorical_data_boston.shape)
print(X_categorical_data_london.shape)

(3371, 16)
(1599, 4)
(23474, 6)


In [ ]:
print(X_categorical_data.head())
print(X_categorical_data_boston.head())
print(X_categorical_data_london.head())

     BuildingType PrimaryPropertyType                 PropertyName  \
0  NonResidential               Hotel         Mayflower park hotel   
1  NonResidential               Hotel              Paramount Hotel   
2  NonResidential               Hotel      5673-The Westin Seattle   
3  NonResidential               Hotel                    HOTEL MAX   
4  NonResidential               Hotel  WARWICK SEATTLE HOTEL (ID8)   

           Address     City State TaxParcelIdentificationNumber Neighborhood  \
0    405 Olive way  Seattle    WA                    0659000030     DOWNTOWN   
1  724 Pine street  Seattle    WA                    0659000220     DOWNTOWN   
2  1900 5th Avenue  Seattle    WA                    0659000475     DOWNTOWN   
3   620 STEWART ST  Seattle    WA                    0659000640     DOWNTOWN   
4    401 LENORA ST  Seattle    WA                    0659000970     DOWNTOWN   

       ListOfAllPropertyUseTypes LargestPropertyUseType  \
0                          Hotel       

## TASK 2.1: make a scatter plot to assess covariance of numerical variables

### IMPORTANT!!

there are going to be some **very** collinear variable - zoom in and chek them out - what are they? You will need to remove variables that are _identical_, because you obviously do not want to imput the data twice in the model - it will not improve the model and it will impact the feature importance analysis.

What else looks like it should be removed?

**Reason about it and explain your choices in the caption of the figure**



In [ ]:
import seaborn as sns
#sns.pairplot(X_numeric_data_boston)

In [ ]:

#sns.pairplot(X_numeric_data_london)

As they do in the original research https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd we should transform some variables into their log values

their original code (which is in R not in python!) is
```
x_seattle$log_total_area=log(x_seattle$gross_floor_area_total_SF)
x_seattle$log_parking_area=log(x_seattle$gross_floor_area_parking_SF)
x_seattle$log_bldg_area=log(x_seattle$gross_floor_area_building_SF)
x_seattle$log_area_1=log(x_seattle$gross_floor_area_property_type_first_SF)

```
the names are a bit different


In [ ]:
# note: recheck missing values
# this is important: after I apply the log function to some variables cause log(0) = -infinity
print("there are {} missing or infinity values in the SEATTLE numerical data".format(X_numeric_data.isnull().sum().sum()))
print("there are {} missing or infinity values in the BOSTON numerical data".format(X_numeric_data_boston.isnull().sum().sum()))
print("there are {} missing or infinity values in the LONDON numerical data".format(X_numeric_data_london.isnull().sum().sum()))

there are 8750 missing or infinity values in the SEATTLE numerical data
there are 839 missing or infinity values in the BOSTON numerical data
there are 0 missing or infinity values in the LONDON numerical data


In [ ]:
# translated to python
X_numeric_data["PropertyGFATotal"] = np.log10(X_numeric_data["PropertyGFATotal"])
X_numeric_data_london['floor_area_SF'] = np.log10(X_numeric_data_london['floor_area_SF'])
X_numeric_data_boston['gross_area_SF'] = np.log10(X_numeric_data_boston['gross_area_SF'])

In [ ]:
print("number of infinite values in SEATTLE:", (np.isinf(X_numeric_data)).sum().sum())
print("number of infinite values in BOSTON:", (np.isinf(X_numeric_data_boston)).sum().sum())
print("number of infinite values in LONDON:", (np.isinf(X_numeric_data_london)).sum().sum())

number of infinite values in SEATTLE: 0
number of infinite values in BOSTON: 0
number of infinite values in LONDON: 0


check what happened to missing values: they may have grown! because  log(0) = -infinity - fix infinities replacing them with NaN

(in in https://github.com/Urban-Informatics-Lab/Open-Data-Benchmarking/blob/master/Lasso_RandomForest.Rmd they had replaced them with 0 but I object to that)

In [ ]:
# this has to return 0 now
X_numeric_data.replace(-np.inf, np.nan, inplace=True)
print("number of infinite values:", (np.isinf(X_numeric_data)).sum().sum())

number of infinite values: 0


In [ ]:
X_numeric_data = X_numeric_data.dropna(axis=1, how='all')
X_numeric_data_boston = X_numeric_data_boston.dropna(axis=1, how='all')
X_numeric_data_london = X_numeric_data_london.dropna(axis=1, how='all')

In [ ]:
print("the numerical SEATTLE data contains (rows, columns) = ", X_numeric_data.shape)
print("there are {} missing values in the SEATTLE numerical data".format(X_numeric_data.isnull().sum().sum()))

print("the numerical BOSTON data contains (rows, columns) = ", X_numeric_data_boston.shape)
print("there are {} missing values in the BOSTON numerical data".format(X_numeric_data_boston.isnull().sum().sum()))

print("the numerical LONDON data contains (rows, columns) = ", X_numeric_data_london.shape)
print("there are {} missing values in the LONDON numerical data".format(X_numeric_data_london.isnull().sum().sum()))

the numerical SEATTLE data contains (rows, columns) =  (3371, 28)
there are 5379 missing values in the SEATTLE numerical data
the numerical BOSTON data contains (rows, columns) =  (1599, 11)
there are 839 missing values in the BOSTON numerical data
the numerical LONDON data contains (rows, columns) =  (23474, 18)
there are 0 missing values in the LONDON numerical data



you can use `KNNImputer` on the numerical variables, but KNNImputer would fail on categorical variables so you are left with NaNs on categorical variables, which you then need to drop before you run the Random Forest


You can impute separately the numerical and categorical variables, the numerical ones with KNNImputer and the categorical ones with SimpleImputer

## TASK 2.2 use Nearest Neighbor Imputer to impute missing values in numerical features

here we are deviating from the paper. NN imputation is simpler and more common, I think it may be more useful to you in general

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3, weights='uniform')
Xn = imputer.fit_transform(X_numeric_data)
Xn = pd.DataFrame(Xn, columns=X_numeric_data.columns)

imputer_boston = KNNImputer(n_neighbors=5, weights='uniform')
Xn_boston = imputer_boston.fit_transform(X_numeric_data_boston)
Xn_boston = pd.DataFrame(Xn_boston, columns=X_numeric_data_boston.columns)

imputer_london = KNNImputer(n_neighbors=5, weights='distance')
Xn_london = imputer_london.fit_transform(X_numeric_data_london)
Xn_london = pd.DataFrame(Xn_london, columns=X_numeric_data_london.columns)

## TASK 2.3 Impute missing categorial variables
you will have to remove Nan values and replace them.

You can use `sklearn.impute.SimpleImputer using strategy="most_frequent"` on all variables, but you will have a naif solution for numerical imputations

I prefer to replace them with "None" or a similar string, for which you can use the fillna method: ```df.fillna("None")``` and leave it at that or with a little more effort you can input values based on nearest neighbors or some other inputation scheme. any choice goes so long as it is well described and justified with words.

In [ ]:
X_categorical_data = X_categorical_data.fillna("None")
X_categorical_data_boston = X_categorical_data_boston.fillna("None")
X_categorical_data_london = X_categorical_data_london.fillna("None")

print(X_categorical_data.shape)
print(X_categorical_data_boston.shape)
print(X_categorical_data_london.shape)


(3371, 16)
(1599, 4)
(23474, 6)


# TASK 3 One Hot Encode the Categorical Variables

Hint: Once you have done the fit and the transformation, which gives you the transformed data according to the new encoding, what you get is normally a sparse matrix. In principle there is an argument `sparse` to `OneHotEncoder` which should prevent this and give you a dense matrix, which is much easier to handle (consider asking me or looking up what is a sparse matrix!). That attribute did not work for me so once I applied the new encoding I converted the result to a dense matrix as
`Xc = Xc.todense()`

Finally, you will need to use a specific method to get the new names of the variables. Its a method of your model (which I called ohe below) and you can find it in this page here https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
X_categorical_data.describe()

,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,TaxParcelIdentificationNumber,Neighborhood,ListOfAllPropertyUseTypes,LargestPropertyUseType,SecondLargestPropertyUseType,ThirdLargestPropertyUseType,YearsENERGYSTARCertified,DefaultData,ComplianceStatus,Outlier
count,3371,3371,3371,3371,3371,3371,3371,3371,3371,3371,3371,3371,3371,3371,3371,3371
unique,8,24,3357,3349,1,1,3263,19,467,57,51,45,66,2,4,3
top,NonResidential,Low-Rise Multifamily,Northgate Plaza,2600 SW Barton St,Seattle,WA,1625049001,DOWNTOWN,Multifamily Housing,Multifamily Housing,None,None,None,False,Compliant,None
freq,1458,984,3,4,3371,3371,8,572,866,1667,1692,2775,3252,3258,3211,3339


In [ ]:
from sklearn.preprocessing import OneHotEncoder
# DELETE
#your code here

ohe = OneHotEncoder(handle_unknown='ignore') #was getting size errors using one
ohe_boston = OneHotEncoder(handle_unknown='ignore')
ohe_london = OneHotEncoder(handle_unknown='ignore')

Xc = ohe.fit_transform(X_categorical_data).todense()
Xc_boston = ohe_boston.fit_transform(X_categorical_data_boston).todense()
Xc_london = ohe_london.fit_transform(X_categorical_data_london).todense()

Xc = pd.DataFrame(Xc, columns=ohe.get_feature_names_out(X_categorical_data.columns))
Xc_boston = pd.DataFrame(Xc_boston, columns=ohe_boston.get_feature_names_out(X_categorical_data_boston.columns))
Xc_london = pd.DataFrame(Xc_london, columns=ohe_london.get_feature_names_out(X_categorical_data_london.columns))


print(Xc.shape)
print(Xc_boston.shape)
print(Xc_london.shape)


(3371, 10717)
(1599, 339)
(23474, 24627)


In [ ]:
# there is a speific method of ohe that allows you to get the names of the features after the encoding is done

feature_name_cat = ohe.get_feature_names_out(X_categorical_data.columns)
feature_name_cat_boston = ohe_boston.get_feature_names_out(X_categorical_data_boston.columns)
feature_name_cat_london = ohe_london.get_feature_names_out(X_categorical_data_london.columns)

print("Seattle Feature Names:", feature_name_cat)
print("Boston Feature Names:", feature_name_cat_boston)
print("London Feature Names:", feature_name_cat_london)


Seattle Feature Names: ['BuildingType_Campus' 'BuildingType_Multifamily HR (10+)'
 'BuildingType_Multifamily LR (1-4)' ... 'Outlier_High outlier'
 'Outlier_Low outlier' 'Outlier_None']
Boston Feature Names: ['reported_Municipal ' 'reported_Voluntary' 'reported_Yes'
 'property_type_Adult Education'
 'property_type_Ambulatory Surgical Center' 'property_type_Aquarium'
 'property_type_Automobile Dealership' 'property_type_Barracks'
 'property_type_College/University' 'property_type_Courthouse'
 'property_type_Distribution Center' 'property_type_Enclosed Mall'
 'property_type_Energy/Power Station' 'property_type_Financial Office'
 'property_type_Fire Station'
 'property_type_Fitness Center/Health Club/Gym'
 'property_type_Food Service'
 'property_type_Hospital (General Medical & Surgical)'
 'property_type_Hotel' 'property_type_Ice/Curling Rink'
 'property_type_Indoor Arena' 'property_type_K-12 School'
 'property_type_Laboratory' 'property_type_Library'
 'property_type_Lifestyle Center'
 'pr

# Task 4 recombine your numerical and categorical imput features and split your data into training and testing

In [ ]:
# this dont need to be the same for you
Xn.shape

(3371, 28)

In [ ]:
# this dont need to be the same for you
Xc.shape

(3371, 10717)

In [ ]:
X = np.hstack([Xn, Xc])
X.shape

Xboston = np.hstack([Xn_boston, Xc_boston])
Xlondon = np.hstack([Xn_london, Xc_london])

In [ ]:
print("The SEATTLE dataset has {} features (!!!)".format(X.shape[1]))
print("The LONDON dataset has {} features (!!!)".format(Xboston.shape[1]))
print("The BOSTON dataset has {} features (!!!)".format(Xlondon.shape[1]))

The SEATTLE dataset has 10745 features (!!!)
The LONDON dataset has 350 features (!!!)
The BOSTON dataset has 24645 features (!!!)


In [ ]:
# make a train and test dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xc, y, test_size=0.3, random_state=42)

X_train_boston, X_test_boston, y_train_boston, y_test_boston = train_test_split(
    Xc_boston, yboston, test_size=0.3, random_state=42)

X_train_london, X_test_london, y_train_london, y_test_london = train_test_split(
    Xc_london, ylondon, test_size=0.3, random_state=42)

# TASK 5 run a random forest REGRESSION model

report the score on the tranining and test data and identify the most important features

(hint: if you see "ValueError: Input contains NaN, infinity or a value too large for dtype('float64')." check
1. np.isnan(Xn).sum().sum() and np.isnan(Xc).sum().sum() both return 0
2. np.isinf(Xn).sum().sum() returns 0
3. all the feature that had very large numbers (> 1e5) have been converted to the log of their values

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_seattle = RandomForestRegressor(
    n_estimators=50, criterion='squared_error', max_depth=None, random_state=42)
rf_boston = RandomForestRegressor(
    n_estimators=50, criterion='squared_error', max_depth=None, random_state=42)
rf_london = RandomForestRegressor(
    n_estimators=50, criterion='squared_error', max_depth=None, random_state=42)


rfseattle = rf_seattle.fit(X_train, y_train)
rfboston = rf_boston.fit(X_train_boston, y_train_boston)
rflondon = rf_london.fit(X_train_london, y_train_london)

In [ ]:
print ("the model test accuracy for SEATTLE is {:.2}".format(rfseattle.score(X_test, y_test)))
print ("the model test accuracy for BOSTON is {:.2}".format(rfboston.score(X_test_boston, y_test_boston)))
print ("the model test accuracy for LONDON is {:.2}".format(rflondon.score(X_test_london, y_test_london)))

In [ ]:
#this is how you see the importance of the features
print("Seattle Feature Importances:", rf.feature_importances_[:10])
print("Boston Feature Importances:", rf_boston.feature_importances_[:10])
print("London Feature Importances:", rf_london.feature_importances_[:10])


In [ ]:
feature_names = Xc.columns
feature_names_boston = Xc_boston.columns
feature_names_london = Xc_london.columns

print("Number of features for Seattle:", len(feature_names))
print("Number of features for Boston:", len(feature_names_boston))
print("Number of features for London:", len(feature_names_london))

In [ ]:
# plot the top ~50 features
import pylab as plt

sorted_idx = np.argsort(rf.feature_importances_)[::-1]  # Sort in descending order

y_ticks = np.arange(0, len(feature_names))
fig, ax = plt.subplots(figsize=(10,20))
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticklabels(feature_names[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
ax.set_ylim(550, len(feature_names))
plt.show()

# Repeat the same process for Boston
sorted_idx_boston = np.argsort(rf_boston.feature_importances_)[::-1]
y_ticks_boston = np.arange(0, len(feature_names_boston))
fig_boston, ax_boston = plt.subplots(figsize=(10, 20))
ax_boston.barh(y_ticks_boston, rf_boston.feature_importances_[sorted_idx_boston])
ax_boston.set_yticks(y_ticks_boston)
ax_boston.set_yticklabels(feature_names_boston[sorted_idx_boston])
ax_boston.set_title("Random Forest Feature Importances (MDI) for Boston")
fig_boston.tight_layout()
plt.show()

# Repeat the same process for London
sorted_idx_london = np.argsort(rf_london.feature_importances_)[::-1]
y_ticks_london = np.arange(0, len(feature_names_london))
fig_london, ax_london = plt.subplots(figsize=(10, 20))
ax_london.barh(y_ticks_london, rf_london.feature_importances_[sorted_idx_london])
ax_london.set_yticks(y_ticks_london)
ax_london.set_yticklabels(feature_names_london[sorted_idx_london])
ax_london.set_title("Random Forest Feature Importances (MDI) for London")
fig_london.tight_layout()
plt.show()

In [ ]:
# CAPTION HERE! How is your model, How is your feature importance results? what can you notice? is it interesting? is it insightful??

# TASK 6 do it for another city! note: the variable names wont be the same


In [ ]:
### I DID BOSTON AND LONDON INLINE IN THE CODES ABOVE

# useful links and methods and functions I used:

```df.dropna()``` to remove observation with missing variables

```df.drop(columns, axis=1)``` to remove features (columns)


```np.log10()``` and ```np.isinf()``` to convert to log space and find infinite values  

`np.hstack([X1, X2])` to concatenate two arrays


one hot encoder https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html and https://stackabuse.com/one-hot-encoding-in-python-with-pandas-and-scikit-learn/


KNN imputer https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html , https://www.analyticsvidhya.com/blog/2020/07/knnimputer-a-robust-way-to-impute-missing-values-using-scikit-learn/, and
https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/  